In [1]:
from pathlib import Path
import json
from src.data_structures import Instance
from src.solvers import ZeroReductor,BaldoML,BaldoGA
from random import shuffle
import pandas as pd
from src.Gurobi import gurobi,SolverConfig,VAR_TYPE
from IPython.display import clear_output
import os

/usr/lib/python3.11/site-packages/torch/__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /build/python-pytorch/src/pytorch-opt/torch/csrc/tensor/python_tensor.cpp:451.)
  _C._set_default_tensor_type(t)


In [2]:
baldo_data_path = Path("baldo_test_data")
results_path = Path("results")

In [3]:
instances = []
for file in baldo_data_path.iterdir():
    instance = Instance.from_file(file)
    instance.name = file.name
    instances.append(instance)
instances.sort(key=lambda x: x.n_items,reverse=False)

In [4]:
solved_files = list(results_path.iterdir())    
for i,instance in enumerate(instances):
    if results_path/instance.name in solved_files:
            continue
    baldo_ml = BaldoML.solve(instance)
    baldo_ga = BaldoGA.solve(instance)
    gurobi_sol = gurobi(instance,SolverConfig(VAR_TYPE.BINARY,heuristic=False,indexes=[],time_limit=600))
    print("BML: " , baldo_ml)
    print("BGA: " , baldo_ga)
    print("Gurobi: " , gurobi_sol)
    print(instance)
    print(i/len(instances))
    clear_output(True)
    fila = {'name': instance.name,
            "BaldoML_of": baldo_ml.o,
            "BaldoML_time": baldo_ml.time,
            "BaldoGA_of": baldo_ga.o,
            "BaldoGA_time": baldo_ga.time,
            "Solver_of": gurobi_sol.o,
            "Solver_time": gurobi_sol.time
            }
    save_file = open(results_path/instance.name,"w")
    json.dump(fila,save_file)
    save_file.close()


Set parameter TimeLimit to value 600
